In [ ]:
# Instalar OpenCV
!pip install opencv-python numpy

In [6]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

# --- 1. CONFIGURACIÓN ---
IMAGE_NAME = "test-image.png"
OUTPUT_DIR = "output_images_full"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 2. CARGAR LA IMAGEN ---
image = cv2.imread(IMAGE_NAME)
if image is None:
    print(f"Error: No se pudo cargar la imagen '{IMAGE_NAME}'.")
else:
    print("Imagen cargada exitosamente.")
    cv2.imwrite(os.path.join(OUTPUT_DIR, "0_original.jpg"), image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # --- 3. FUNCIONES PARA LOS FILTROS ---
    def convert_to_hsv(img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    def convert_to_lab(img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    def convert_to_monochrome(img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    def simulate_daltonism(img_rgb):
        protanopia_matrix = np.array([[0.567, 0.433, 0], [0.558, 0.442, 0], [0, 0.242, 0.758]])
        img_float = img_rgb.astype(np.float32) / 255.0
        dalton_float = np.dot(img_float, protanopia_matrix.T)
        dalton_rgb = (np.clip(dalton_float, 0, 1) * 255).astype(np.uint8)
        return cv2.cvtColor(dalton_rgb, cv2.COLOR_RGB2BGR)

    def apply_temperature_filter(img, warmth):
        # Aumentar o disminuir el rojo y azul para simular temperatura
        # 'warmth' puede ser un valor como 30 (cálido) o -30 (frío)
        b, g, r = cv2.split(img.astype("int16"))
        r = np.clip(r + warmth, 0, 255)
        b = np.clip(b - warmth, 0, 255)
        # Ensure all channels are uint8 before merging and merge in BGR order
        return cv2.merge([b.astype("uint8"), g.astype("uint8"), r.astype("uint8")])


    # --- 4. APLICAR Y GUARDAR TODO ---
    # Conversiones
    image_hsv = convert_to_hsv(image)
    image_lab = convert_to_lab(image)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "1a_conversion_hsv.jpg"), image_hsv)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "1b_conversion_lab.jpg"), image_lab)

    # Canales
    h, s, v = cv2.split(image_hsv)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "2a_canal_h.jpg"), h)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "2b_canal_s.jpg"), s)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "2c_canal_v.jpg"), v)

    # Filtros
    image_mono = convert_to_monochrome(image)
    image_dalton = simulate_daltonism(image_rgb)
    image_warm = apply_temperature_filter(image, 40) # Filtro cálido
    image_cold = apply_temperature_filter(image, -40) # Filtro frío
    cv2.imwrite(os.path.join(OUTPUT_DIR, "3a_filtro_monocromo.jpg"), image_mono)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "3b_simulacion_daltonismo.jpg"), image_dalton)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "3c_filtro_temperatura_calida.jpg"), image_warm)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "3d_filtro_temperatura_fria.jpg"), image_cold)

    print("\n--- ¡Proceso completado! Todas las imágenes han sido guardadas. ---")

# --- 5. BONUS: FUNCIÓN INTERACTIVA CON WIDGETS (VERSIÓN COMPLETA) ---
import ipywidgets as widgets
from IPython.display import display, clear_output

# Crear un diccionario completo para acceder a TODAS las imágenes generadas
images = {
    # Original
    "Original": image,

    # Separador (un truco para organizar el menú)
    "--- Conversiones Completas ---": None,
    "Conversión a HSV": image_hsv,
    "Conversión a LAB": image_lab,

    # Separador
    "--- Canales Individuales (HSV) ---": None,
    "Canal H (Tono)": h,
    "Canal S (Saturación)": s,
    "Canal V (Brillo/Valor)": v,

    # Separador
    "--- Filtros y Simulaciones ---": None,
    "Monocromo (Escala de Grises)": image_mono,
    "Simulación Daltonismo": image_dalton,
    "Filtro Cálido": image_warm,
    "Filtro Frío": image_cold
}

# 1. Crear el menú desplegable
dropdown = widgets.Dropdown(
    options=images.keys(),
    description='Elige una vista:',
    value='Original'
)

# 2. Crear un área de salida
output_area = widgets.Output()

# 3. Definir la función que se ejecuta cuando cambia el menú
def on_view_change(change):
    selected_key = change['new']
    image_to_show = images[selected_key]

    # Limpiar el área de salida
    with output_area:
        clear_output(wait=True)

        # Mostrar la imagen solo si no es un separador (que es 'None')
        if image_to_show is not None:
            print(f"Mostrando: {selected_key}")
            cv2_imshow(image_to_show)

# 4. Vincular la función al menú
dropdown.observe(on_view_change, names='value')

# 5. Mostrar los widgets
print("\n--- Menú Interactivo de Visualización (Completo) ---")
display(dropdown, output_area)

# 6. Mostrar la imagen inicial
on_view_change({'new': 'Original'})

Imagen cargada exitosamente.

--- ¡Proceso completado! Todas las imágenes han sido guardadas. ---

--- Menú Interactivo de Visualización (Completo) ---


Dropdown(description='Elige una vista:', options=('Original', '--- Conversiones Completas ---', 'Conversión a …

Output()